In [1]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

def agregatedComputation(stock_data_output_list):
    list_company_name = []
    list_index_operated = []
    list_number_aproved_appointments = []
    list_key_date = []
    list_key_date_value_average_ranking = []
    list_coeficient = []

    file_path = '../Datos/FDA Aproval.xlsx'
    all_sheets_FDA = pd.ExcelFile(file_path).sheet_names
    FDA_index = ["Russell2000"]*13 + ["S&P500"]*11
    
    for i in range(0,len(all_sheets_FDA)):
        print(i)
        df_plot = stock_data_output_list[i]

        #Impact Analysis in Price
        matrix_rows = list(df_plot[df_plot["FDA Date"] == 1]["Date"])
        matrix_cols = np.arange(-10, 2)
        perc_change = np.diff(np.array(df_plot["Close"])) / np.array(df_plot["Close"])[:-1] * 100
        perc_change_final = np.insert(perc_change, 0, 0)
        df_output_price = pd.DataFrame(index=range(len(matrix_rows)), columns=range(len(matrix_cols)))


        for j in np.arange(0,len(matrix_rows)):
            date_index = np.where(df_plot['Date'] == matrix_rows[j])
            selected_index = np.arange(date_index[0]-10,date_index[0]+2)
            selected_changes = perc_change_final[selected_index]
            order_of_changes = np.argsort(selected_changes)
            df_output_price.iloc[j] = order_of_changes

    
        matrix_cols = ["Day {}".format(i) for i in matrix_cols]
        df_output_price.columns = matrix_cols
        df_output_price = df_output_price.set_index(pd.Index(matrix_rows))
        average_success = df_output_price.mean()
        selected_date = np.argmin(df_output_price.mean()) - 10

        ###Preference Plot
        Categories = list(df_output_price.columns)
        Average_ranking_position = list(average_success)
        min_value_index = Average_ranking_position.index(min(Average_ranking_position))




        ###HeatMap
        data = {'Day -10': list(df_output_price.iloc[:,0]),
                'Day -9': list(df_output_price.iloc[:,1]),
                'Day -8': list(df_output_price.iloc[:,2]),
                'Day -7': list(df_output_price.iloc[:,3]),
                'Day -6': list(df_output_price.iloc[:,4]),
                'Day -5': list(df_output_price.iloc[:,5]),
                'Day -4': list(df_output_price.iloc[:,6]),
                'Day -3': list(df_output_price.iloc[:,7]),
                'Day -2': list(df_output_price.iloc[:,8]),
                'Day -1': list(df_output_price.iloc[:,9]),
                'Day 0': list(df_output_price.iloc[:,10]),
                'Day 1': list(df_output_price.iloc[:,11])}

        df = pd.DataFrame(data)
        df = df.set_index(pd.Index([timestamp.strftime('%Y-%m-%d') for timestamp in matrix_rows]))


        list_changed_series = perc_change_final
        FDA_date = list(df_plot["FDA Date"])
        modified_FDA_date = FDA_date[abs(selected_date):] + [0] * abs(selected_date)

        X_set = {'Y_t-5': list_changed_series[1:-5],
                'Y_t-4':  list_changed_series[2:-4],
                'Y_t-3':  list_changed_series[3:-3],
                'Y_t-2':  list_changed_series[4:-2],
                'Y_t-1':  list_changed_series[5:-1],
                'FDA_YN': modified_FDA_date[6:]}
        X_df = pd.DataFrame(X_set)

        X_input = X_df.values
        Y_input = list_changed_series[6:]

        train_test_break = date_index - abs(selected_date) - 40
        X_train = X_input[0:int(train_test_break[0])]
        y_train = Y_input[0:int(train_test_break[0])]
        X_test  = X_input[int(train_test_break[0]):]
        y_test  = Y_input[int(train_test_break[0]):]

        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        FDA_coef = model.coef_[5]


        ##Model Without Coefficient
        X_set_2 = {'Y_t-5': list_changed_series[1:-5],
                'Y_t-4':  list_changed_series[2:-4],
                'Y_t-3':  list_changed_series[3:-3],
                'Y_t-2':  list_changed_series[4:-2],
                'Y_t-1':  list_changed_series[5:-1]}
        X_df_2 = pd.DataFrame(X_set_2)

        X_input_2 = X_df_2.values
        Y_input_2 = list_changed_series[6:]
    
        train_test_break_2 = date_index - abs(selected_date) - 40
        X_train_2 = X_input_2[0:int(train_test_break_2[0])]
        y_train_2 = Y_input_2[0:int(train_test_break_2[0])]
        X_test_2  = X_input_2[int(train_test_break_2[0]):]
        y_test_2  = Y_input_2[int(train_test_break_2[0]):]

        model_2 = LinearRegression()
        model_2.fit(X_train_2, y_train_2)
        y_pred_2 = model_2.predict(X_test_2)


        X_axis = list(df_plot["Date"])[-(len(y_test)+15):]
        Y_real = list(y_train[-15:]) + list(y_test)
        Y_predicted_FDA = list(y_train[-15:]) + list(y_pred)
        Y_predicted_WO_FDA = list(y_train[-15:]) + list(y_pred_2)

        company_name = all_sheets_FDA[i]
        list_company_name.append(company_name)
        index_operated = FDA_index[i]
        list_index_operated.append(index_operated)
        number_aproved_appointments = sum(df_plot['FDA Date'])
        list_number_aproved_appointments.append(number_aproved_appointments)
        key_date = selected_date
        list_key_date.append(key_date)
        key_date_value_average_ranking = min(df_output_price.mean())
        list_key_date_value_average_ranking.append(key_date_value_average_ranking)
        coeficient = FDA_coef
        list_coeficient.append(coeficient)
    
    final_output_list = [list_company_name,list_index_operated,list_number_aproved_appointments,list_key_date,
                         list_key_date_value_average_ranking,list_coeficient]
    return(final_output_list)